In [15]:
import secrets
import hashlib

In [16]:
def random_binary_generator(n):
    # Gets integer n,
    # Generates n random bits and returns it 
    # (Without the 0b)
    random_bits = secrets.randbits(n)
    binary_string = bin(random_bits)[2:]
    binary_string = binary_string.zfill(n)

    return binary_string

In [17]:
def count_one_elements(binary_number):
    # Gets a binary number,
    # Counts '1' elements in binary number and returns it
    count = 0
    for digit in binary_number:
        if digit == '1':
            count += 1
    return count

def rotate_left(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[shift_amount:] + binary_number[:shift_amount]
    return rotated_binary

def rotate_right(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[-shift_amount:] + binary_number[:-shift_amount]
    return rotated_binary

In [18]:
def binary_xor(*args):
    # Gets anu amount of binary numbers, 
    # (converts them to same lenth if they are not)
    # Calculates XOR of them and then returns it

    max_length = max(len(binary_string) for binary_string in args)
    padded_args = [binary_string.zfill(max_length) for binary_string in args]
    xor_binary_string = ''.join(str(sum(int(bit) for bit in bits) % 2) for bits in zip(*padded_args))

    return xor_binary_string

In [19]:
def hash_binary(binary_string):
    # Gets a binary number,
    # returns hash of it in hex
    decimal_number = int(binary_string, 2)
    byte_data = decimal_number.to_bytes((decimal_number.bit_length() + 7) // 8, byteorder='big')
    sha1_hash = hashlib.sha1(byte_data).hexdigest()

    sha1_hash = '0x' + sha1_hash
    return sha1_hash

## IDs of nodes

In [20]:
# Should be sent from Reader => Supply
# Msg 3
PP_bin = '0101001111010001001010111111011101000110110100000011000100001110010111000001100110101110100000101010011101101010101000100101110111010101110001000110011101001101'
PQ_bin = '0111101000101100110101010010110011111110100010001011101000101001001010101100011110100100110011100010011001101010000110101101001010111011101011110111110111111100'
Reader_check_hex = '0x5df5d19d9d365ce4788b762b95c789034b3cab5e'
TIR_prime_bin = '10101110111011001011000001101000'

In [21]:
# Should be shared
IDNt_bin = '0010101111000011001000000010100100110011000011001010000101010011001010100101110001011000101100111110110010001111100011000111101111111111101100101011110001110010'
IDNr_bin = '0010110111001000001111000010111111100010010100100111101001101110100101011000100001010100000110000001011010010101001010110110101010110000011011110000111001110000'
IDNs_bin = '1010001111000111111001111100111111001000111110101010001011000001110111000101011110101110100001011000111001000000001101101101111011110001010001010101000111000001'
BalN_bin = '1000110111111000011000100100101111111011001101100010000110111101101011110000110001001001001011011101100000010000001001101111111000010110111100000001101010011011'
BS_bin = '1001101011100011111010110101101111000000010110111111101011111100110100001001100101010111010101010010110001010010010111100110101101100000011000011001111110000000'

In [22]:
# YRS
# IDNs_bin || BS_bin || IDNr_bin (? is this ok ?)
pre_YRS = IDNs_bin + BS_bin + IDNr_bin
YRS_hex = hash_binary(pre_YRS)

In [23]:
TIS_bin = random_binary_generator(32)
S0_bin = random_binary_generator(160)

In [24]:
# SE
# IDNs_bin ^ YRS_hex
SE_left_2_bin = binary_xor(IDNs_bin, bin(int(YRS_hex, 16))[2:])
SE_left_bin = rotate_left(TIS_bin, count_one_elements(SE_left_2_bin))

SE_right_bin = rotate_left(S0_bin, count_one_elements(bin(int(YRS_hex, 16))[2:]))

SE_bin = binary_xor(SE_left_bin, SE_right_bin)

In [25]:
# SF
SF_left_bin = rotate_left(S0_bin, count_one_elements(IDNs_bin))
SF_right_bin = rotate_left(TIS_bin, count_one_elements(bin(int(YRS_hex, 16))[2:]))

SF_bin = binary_xor(SF_left_bin, SF_right_bin)

In [26]:
# IDNt_bin ^ BalN_bin ^ S0_bin ^ IDNs_bin
pre_TKST = binary_xor(IDNt_bin, BalN_bin, S0_bin, IDNs_bin)
TKST_hex = hash_binary(pre_TKST)

In [27]:
pre_SG = bin(int(TKST_hex, 16))[2:] + S0_bin + BalN_bin
SG_hex = hash_binary(pre_SG)

In [28]:
msg4 = {
    "SE_bin" : SE_bin,
    "SF_bin" : SF_bin,
    "SG_hex" : SG_hex,
    "TIS_bin" : TIS_bin
}

print(f"SE_bin = '{SE_bin}'")
print(f"SF_bin = '{SF_bin}'")
print(f"SG_hex = '{SG_hex}'")
print(f"TIS_bin = '{TIS_bin}'")

SE_bin = '0000001111101001111000010101001011010001110000111011010001101011100001110110100110111000000011010010111001111010010101101000100111110110000111110001100011110111'
SF_bin = '1110000101010010110100011100001110110100011010111000011101101001101110000000110100101110011110100101011010001001011110101111011110101110100111010011100101101011'
SG_hex = '0xe1595ed7cefbd88dae1f4476f63d57a60a3117d5'
TIS_bin = '10001100111010000011101010000010'


In [29]:
S0_bin

'1010010111001111010010101101000100101111010111101110010001001110101000000111110100111100001010100101101000111000011101101000110101110000111011010011011100000001'